Test with Acropora abrolhosensis https://www.marinespecies.org/aphia.php?p=taxdetails&id=288182
aphiaID = 288182  
urn = urn:lsid:marinespecies.org:taxname:288182

In [1]:
import json
from urllib.request import urlopen
import urllib.parse
import pandas as pd

Build OBIS API query

In [2]:
base = 'https://api.obis.org/v3/'

aphiaID = '342218'

fields = ['decimalLatitude','decimalLongitude','occurrenceID','id',
          'datasetName','dataset_id','eventDate','species','aphiaID',
          'date_year']

f = "%2C".join([f"{v}" for v in fields])

# define a polygon
# bound it somehow to between 30 deg N and S latitude
#geometry=urllib.parse.quote("POLYGON ((-180 -30, 180 -30, 180 30, -180 30, -180 -30))")

#depth = [0, 30]

# How many responses do you want back? Max=10000
size = 10000

#occurrence_id = 'occurrence?taxonid={}&startdepth={}&enddepth={}&geometry={}&size={}'.format(aphiaID,depth[0],depth[1],geometry,size)
occurrence_id = 'occurrence?taxonid={}&size={}'.format(aphiaID,size)


#taxon_id = 'taxon/{}'.format(aphiaID)
#taxon_sn = 'taxon/{}'.format(scientificname)

url = '{}{}'.format(base,occurrence_id)

url

'https://api.obis.org/v3/occurrence?taxonid=342218&size=10000'

Download the data from OBIS API into pandas DataFrame

In [3]:
response= urlopen(url)

data_json = json.loads(response.read())

data = pd.DataFrame(data_json['results'])

size = data_json['total']

print("Found %i records." % size)
print("See in mapper at: %s" % '{}{}#'.format('https://mapper.obis.org/?',occurrence_id.replace("occurrence?","")))

Found 221 records.
See in mapper at: https://mapper.obis.org/?taxonid=342218&size=10000#


Save DataFrame as GeoDataFrame

In [4]:
import geopandas as gpd

gdf = gpd.GeoDataFrame(
    data, geometry=gpd.points_from_xy(data.decimalLongitude, data.decimalLatitude),
    crs="EPSG:4326")

Bin to 1 degree cells

bin to one degree cells across globe

In [5]:
import numpy as np
import shapely

# total area for the grid
xmin, ymin, xmax, ymax= [-180., -90., 180., 90]
# how many cells across and down
n_cells=360
cell_size = (xmax-xmin)/n_cells
# projection of the grid
#crs = "+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs"
# create the cells in a loop
grid_cells = []
for x0 in np.arange(xmin, xmax+cell_size, cell_size ):
    for y0 in np.arange(ymin, ymax+cell_size, cell_size):
        # bounds
        x1 = x0-cell_size
        y1 = y0+cell_size
        grid_cells.append( shapely.geometry.box(x0, y0, x1, y1)  )
        
cell = gpd.GeoDataFrame(grid_cells, columns=['geometry'], 
                                 crs="EPSG:4326")
    
# ax = gdf.plot(markersize=.1, figsize=(12, 8),)# column='dob', cmap='jet')
# plt.autoscale(False)
# cell.plot(ax=ax, facecolor="none", edgecolor='grey')
# ax.axis("off")

merged = gpd.sjoin(gdf, cell, how='left', op='within')

# make a simple count variable that we can sum
merged['n_occur'] = 1
# Compute stats per grid cell -- aggregate fires to grid cells with dissolve
dissolve = merged.dissolve(by="index_right", aggfunc="count")
# put this into cell
cell.loc[dissolve.index, 'n_occur'] = dissolve.n_occur.values

cell = cell[~cell['n_occur'].isna()]

cell['area (km2)'] = cell.to_crs(epsg=3763).area/(1000**2)

cell['density (#/km2)'] = cell['n_occur']/cell['area (km2)']

C:\Users\Mathew.Biddle\programs\Anaconda3\envs\globe\lib\site-packages\IPython\core\interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [6]:
cell[['n_occur','density (#/km2)']]

,n_occur,density (#/km2)
226,1.0,0.000112
408,1.0,0.000110
590,3.0,0.000322
1313,1.0,0.000108
2226,1.0,0.000092
...,...,...
64844,1.0,0.000111
65020,1.0,0.000121
65021,1.0,0.000119
65206,1.0,0.000110


Now make the plot using geopandas built in plot

Use geoplot

In [8]:
import geoplot
import geoplot.crs as gcrs
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER


norm = mpl.colors.Normalize(vmin=0, vmax=0.000142)

#projection=gcrs.Mollweide(central_longitude=-160)
projection = gcrs.PlateCarree(central_longitude=0.0)


world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world.dissolve()

#fig, ax = plt.subplots(figsize=(16,16), facecolor='white', subplot_kw={'projection':projection})

# ax = geoplot.polyplot(
#     world,
#     figsize=(16,16),
#     facecolor='lightgray',
#     edgecolor=None,
#     projection=projection,
#    # ax=ax,
# )

# divider = make_axes_locatable(ax)
# cax = divider.append_axes("bottom", size="5%", pad=0.1)

#cax = fig.add_axes([0.125, 0.08, 0.775, 0.03])

# cell[~cell['n_occur'].isna()]

#cell = cell[~cell['n_occur'].isna()]



ax = geoplot.choropleth(cell, 
                   hue='density (#/km2)', 
                   #edgecolor='black', 
                   projection=projection, 
                   #ax=ax,
                   cmap='Spectral_r',
#                    legend=True,
#                    legend_kwargs={'orientation': 'horizontal',
#                                   #'cax':cax,
#                                   'label':"occurrence (count)",
#                                  },
                   norm=norm, 
                   figsize=(160, 80),
                  )


geoplot.polyplot(
    world,
   # figsize=(16,16),
    facecolor='black',
    edgecolor=None,
    projection=projection,
    ax=ax,
)

geoplot.pointplot(gdf, ax=ax, projection=projection, color='black', s=.5)

# import cartopy.crs as ccrs

# gl = ax.gridlines(draw_labels={"bottom": "x", "left": "y"}, crs=ccrs.PlateCarree())
# gl.ylocator = mticker.FixedLocator(range(-90,91,30)) #-75, -50, -25, 0, 25, 50, 75, 90])
# gl.xlocator = mticker.FixedLocator(range(-180,181,30)) #-75, -50, -25, 0, 25, 50, 75, 90])
# gl.top_labels = False
# gl.bottom_labels = True
# gl.left_labels = False
# gl.right_labels = True
# gl.xformatter = LONGITUDE_FORMATTER
# gl.yformatter = LATITUDE_FORMATTER

#ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)


#ax.outline_patch.set_visible(True)
ax.set_global(); 

#order = [0,2,1,3]
#extent = [cell.total_bounds[i] for i in order]

#ax.set_extent(extent)

fname='images/map_obis_2010.jpg'

plt.savefig(fname, dpi='figure', format='jpg',
        bbox_inches='tight', pad_inches=0.0,
        facecolor='auto', edgecolor='auto',
        backend=None, #transparent=True,
       )

In [22]:
import geoplot
import geoplot.crs as gcrs
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

norm = mpl.colors.Normalize(vmin=0, vmax=0.000142)

#projection=gcrs.Mollweide(central_longitude=-160)
projection = gcrs.PlateCarree(central_longitude=0.0)


world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world.dissolve()

#fig, ax = plt.subplots(figsize=(16,16), facecolor='white', subplot_kw={'projection':projection})

# ax = geoplot.polyplot(
#     world,
#     figsize=(16,16),
#     facecolor='lightgray',
#     edgecolor=None,
#     projection=projection,
#    # ax=ax,
# )

# divider = make_axes_locatable(ax)
# cax = divider.append_axes("bottom", size="5%", pad=0.1)

#cax = fig.add_axes([0.125, 0.08, 0.775, 0.03])

# cell[~cell['n_occur'].isna()]

#cell = cell[~cell['n_occur'].isna()]


fig, ax = plt.subplots(figsize=(160, 80), 
                       #facecolor='black', 
                       subplot_kw={'projection':projection})

ax.set_facecolor('black')

geoplot.choropleth(cell, 
                   hue='density (#/km2)', 
                   #edgecolor='black', 
                   projection=projection, 
                   ax=ax,
                   cmap='Spectral_r',
#                    legend=True,
#                    legend_kwargs={'orientation': 'horizontal',
#                                   #'cax':cax,
#                                   'label':"occurrence (count)",
#                                  },
                   norm=norm, 
#                   figsize=(160, 80),
                  )


geoplot.polyplot(
    world,
   # figsize=(16,16),
    facecolor='grey',
    edgecolor='black',
    projection=projection,
    ax=ax,
)

#geoplot.pointplot(gdf, ax=ax, projection=projection, color='black', s=.5)

# import cartopy.crs as ccrs

# gl = ax.gridlines(draw_labels={"bottom": "x", "left": "y"}, crs=ccrs.PlateCarree())
# gl.ylocator = mticker.FixedLocator(range(-90,91,30)) #-75, -50, -25, 0, 25, 50, 75, 90])
# gl.xlocator = mticker.FixedLocator(range(-180,181,30)) #-75, -50, -25, 0, 25, 50, 75, 90])
# gl.top_labels = False
# gl.bottom_labels = True
# gl.left_labels = False
# gl.right_labels = True
# gl.xformatter = LONGITUDE_FORMATTER
# gl.yformatter = LATITUDE_FORMATTER

#ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)


#ax.outline_patch.set_visible(True)
ax.set_global(); 

#order = [0,2,1,3]
#extent = [cell.total_bounds[i] for i in order]

#ax.set_extent(extent)

fname='images/map_obis_1990.jpg'

plt.savefig(fname, dpi='figure', format='jpg',
        bbox_inches='tight', pad_inches=0.0,
        facecolor='auto', edgecolor=None,
        backend=None, #transparent=True,
       )